In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [5]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3

In [6]:
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

# one for long-term memories and one for short-term memories
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))

for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

    
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward0>)
(tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward0>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<StackBackward0>))


In [7]:
import numpy as np

In [8]:
freq1, freq2, offsets1, offsets2 = np.random.rand(4, 3, 1)

freq1, freq2, offsets1, offsets2

(array([[0.46408451],
        [0.27627713],
        [0.20421162]]),
 array([[0.89428972],
        [0.2055513 ],
        [0.35796314]]),
 array([[0.19821972],
        [0.01272278],
        [0.80092854]]),
 array([[0.36010825],
        [0.85061172],
        [0.4581304 ]]))

In [14]:
# Define the dataset

import torch
import numpy as np

# Generate a synthetic time series dataset
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  # Wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20))  # Wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)  # Noise
    return series[..., np.newaxis].astype(np.float32)  # ... for compatibility

# Example usage
n_steps = 50
series = generate_time_series(1000, n_steps + 1)
X_train, y_train = series[:, :n_steps], series[:, -1]
X_train, y_train = torch.from_numpy(X_train), torch.from_numpy(y_train)

In [15]:
series.shape

(1000, 51, 1)

In [16]:
# Define the seq-to-seq LSTM

class LSTMModel(torch.nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = torch.nn.LSTM(input_size, hidden_layer_size)
        self.linear = torch.nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]


In [17]:
model = LSTMModel(input_size=1, hidden_layer_size=100, output_size=1)
loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 150
for i in range(epochs):
    for seq, labels in zip(X_train, y_train):
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))

        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    if i%25 == 0:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

epoch:   0 loss: 0.02094631
epoch:  25 loss: 0.01171473
epoch:  50 loss: 0.00474951
epoch:  75 loss: 0.00081661
epoch: 100 loss: 0.00016306
epoch: 125 loss: 0.00087019


In [19]:
# evaluation
# Assuming you have test data (X_test, y_test)
with torch.no_grad():
    preds = []
    for seq in X_train:
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                             torch.zeros(1, 1, model.hidden_layer_size))
        preds.append(model(seq).item())
        
